In [91]:
# menampilkan info deskripsi dataset
data_review.describe()

,label
count,10000.000000
mean,4.764600
std,0.712066
min,1.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


Dari hasil deskripsi review data, yang berjumlah 10.000 memiliki standard deviasi  0,71. Standar deviasi merupakan akar dari varian, sehingga semakin besar nilai standar deviasi maka data sampel semakin menyebar (bervariasi) dari rata-ratanya. 

In [92]:
# menampilkan info dataset
data_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   label     10000 non-null  int64 
 1   text      10000 non-null  object
 2   Sentimen  10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [93]:
#memeriksa duplicate data
data_duplicate = data_review[data_review['text'].duplicated()]
print(f'No. of duplicate reviews on train data: {data_duplicate.shape[0]}')

No. of duplicate reviews on train data: 0


Data yang digunakan sudah terbilang baik, karena tidak terdapat data yang memiliki duplikat dan tidak ada data yang bernilai null

### Preprocessing data

Pra Pemrosesan teks dilakukan untuk menghilangkan data yang tidak dibutuhkan ataupun data yang terdapat dalam teks yang tidak sesuai dengan proses yang dibutuhkan. 

In [94]:
def NER(review):
    for i in range(len(review)):
        text = review.text.iloc[i]
        for sent in nltk.sent_tokenize(text):
            for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
                if hasattr(chunk, 'label') and chunk.label:
                    if chunk.label() == 'ORGANIZATION' or  chunk.label() == 'PERSON' or  chunk.label() == 'DATE' or  chunk.label() == 'LOCATION':
                        name_value = ' '.join(child[0] for child in chunk.leaves())
                        text = text.replace(name_value, "")
                        review.text.iloc[i] = text
    return review

NER (Named-Entity Recognition) digunakan untuk mencari dan mengelompokkan entitas dalam teks ke dalam kategori yang ditetapkan, misalnya nama orang, organisasi, lokasi, ekspresi, dll. 

In [95]:
def case_folding(review):
    for i in range(len(review)):
        text = review.text[i].lower()
        review.text.iloc[i] = text 
    return review

Case folding digunakan untuk mengkonversi keseluruhan teks huruf kapital (uppercase) pada sebuah kalimat menjadi huruf kecil (lowercase) serta menghilangkan seluruh karakter yang dianggap tidak valid seperti angka, tanda baca, dan Uniform Resources Locator (URL). 

In [96]:
def remove_punctuation(review):
    remove = string.punctuation
    for i in range(len(review)):
        for kata in remove:
            text = review.text[i].replace(kata,"")
            review.text.iloc[i] = text 
    return review

Remove punctuation digunakan untuk menghapus semua tanda baca dari string atau ulasan pelanggan.

In [97]:
def stop_removal(review):
    from nltk.tokenize import sent_tokenize, word_tokenize
    cachedStopWords = set(stopwords.words("english"))
    for i in range(len(review)):
        text = review.text.iloc[i]
        teks =" ".join([word for word in text.split() if word not in cachedStopWords])
        review.text.iloc[i] = teks
    return review

Stopwords removal bertujuan untuk mengurangi kata yang kurang penting, seperti kata yang, di, ke, dalam, dan, dengan, ini, itu, untuk, dan lain sebagainya, sehingga mempermudah dan mempercepat pengolahan dokumen. 

In [98]:
def stemming(review):
    ps = PorterStemmer()
    for i in range(len(review)):
        text = review.text.iloc[i]
        text = ps.stem(text)
        review.text.iloc[i] = text
    return review

Stemming digunakan untuk menghilangkan imbuhan-imbuhan seperti awalan kata (prefixes), sisipan kata (infixes), akhiran kata (suffixes) serta awalan dan akhiran kata (confixes) pada kata turunan. yang terdapat dalam kata.

In [99]:
def lemmatization (review):
    lm = WordNetLemmatizer()
    for i in range(len(review)):
        text = review.text.iloc[i]
        text = lm.lemmatize(text)
        review.text.iloc[i] = text
    return review

In [100]:
def preprocessing_data(review):
    hasil_ner = NER(review)
    hasil_case_folding = case_folding(hasil_ner)
    hasil_remove_punctuation = NER(review)
    hasil_stop_removal = stop_removal(hasil_remove_punctuation)
    hasil_stemming = stemming( hasil_stop_removal)
    hasil_lemmatization = lemmatization(hasil_stemming)
    return hasil_lemmatization

Pada program berikut merupakan penerapan semua fungsi pemrosesan data yang telah didefinisikan sebelumnya. Hasil yang akan ditampilkan adalah hasil lematisasi karena lematisasi adalah proses terakhir yang menampung seluruh output tahap-tahap sebelumnya kemudian kemudian menjadi input tahap lematisasi lalu diproses dan menghasilkan output yaitu hasil dari tahap lematisasi tersebut. 

In [101]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pebri Sangmajadi
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Pebri
[nltk_data]     Sangmajadi S\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Pebri Sangmajadi
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [102]:
clean_data = preprocessing_data(data_review)

C:\Users\Pebri Sangmajadi S\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [103]:
import re

pattern=r'[^a-zA-Z]'

for i in range(len(clean_data)):
    clean_data['text'].iloc[i] = re.sub(pattern,' ', clean_data['text'].iloc[i], flags=re.MULTILINE)
clean_data.head(10)

C:\Users\Pebri Sangmajadi S\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,label,text,Sentimen
0,5,looks ok like durable use recommend others w...,Positive
1,5,tried current powerful depending setting dar...,Positive
2,5,received week looks smaller expected can t w...,Positive
3,5,thanks works describe complaints really ex...,Positive
4,5,fast delivery considering it s overseas tried ...,Positive
5,5,fast delivery good servic,Positive
6,5,got order came well packaged yet try looks go...,Positive
7,5,items received nice box used yet hopefully w...,Positive
8,5,received good condition tried far good bad,Positive
9,1,item doesn t work asked send refund show n...,Negative


Perintah di atas bertujuan untuk menampilkan data hasil preprocessing yang sudah dilakukan sebelumnya. Berdasarkan hasil preprocessing telah didapatkan data yang sudah bersih, data hasi pra pemrosesan selanjutnya akan disimpan ke file baru yaitu file dengan format excel.

In [104]:
clean_data.text

0       looks ok  like durable  use recommend others w...
1       tried  current powerful depending setting  dar...
2       received week  looks smaller expected  can t w...
3       thanks    works describe complaints  really ex...
4       fast delivery considering it s overseas tried ...
                              ...                        
9995                          hi product work properly   
9996    best purchase ever   never regret item came me...
9997                  suction good  fall  happy purchase 
9998              goooood suction fast delivery   appreci
9999                received good condition  yet try out 
Name: text, Length: 10000, dtype: object

In [105]:
hasil_preprocessing= data_review.to_excel('data_setelah_preprocessing.xlsx', encoding='utf-8')
new_data = pd.read_excel('./data_setelah_preprocessing.xlsx')

### Feature Extraction